# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [1]:
# Imports
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import string
import re
import numpy as np
import math

# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [2]:
#TODO: implement!
url = 'https://raw.githubusercontent.com/alch00001/NNTI-WS2021-NLP-Project/main/data/hindi_hatespeech.tsv'
data = pd.read_csv(url, sep='\t')
data.astype("string")

#Development set
development = data.loc[0:99].copy()
development.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",NOT,NONE,NONE
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",NOT,NONE,NONE
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,NOT,NONE,NONE


## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [3]:
#TODO: implement!

#remove usernames
development['text']=development['text'].str.replace('@[\w]+','')
#remove puncutation
development['text']=development['text'].str.replace('[^\w\s]','')
#convert to lowercase
development['text'] = development['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#remove numbers
development['text'] = development['text'].str.replace('\d+', '')

#import and remove stopwords
stopurl = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-hi/master/stopwords-hi.txt'
stopwords = pd.read_csv(stopurl, sep='\t')
stopwords.astype("string")
development['text'] = development['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

#remove emojis, script taken from github
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
development['text'] = development['text'].apply(lambda x: remove_emoji(x))

development.head(10)

,text_id,text,task_1,task_2,task_3
0,hasoc_hi_5556,बगलदश क शनदर वपस भरत क रन पर रक indvban cwc,NOT,NONE,NONE
1,hasoc_hi_5648,सब रड नच दखन म वयसत जस ह कई शतदत क सथ कछ हग सब...,HOF,PRFN,UNT
2,hasoc_hi_164,तम जस हरमय क लए बस जत क कम ह शकर कर अभ तमहर लच...,HOF,PRFN,TIN
3,hasoc_hi_3530,बजप mla आकश वजयवरगय जल स रह जमनत मलन क खश म एक...,NOT,NONE,NONE
4,hasoc_hi_5206,चमक बखर वधनसभ परसर म आरजड क परदरशन तजसव यदव नद...,NOT,NONE,NONE
5,hasoc_hi_5121,मबई म बरश स लग क कफ समसय ह रह ह,NOT,NONE,NONE
6,hasoc_hi_7142,ahmeds dad beta aaj teri mammy kyu nahi baat k...,NOT,NONE,NONE
7,hasoc_hi_4321,लख मसलमन उरस म अजमर क दरगह पर आत ह सरफ पलस वल ...,NOT,NONE,NONE
8,hasoc_hi_4674,do mahashaktiyan mili hain charo taraf khusi k...,NOT,NONE,NONE
9,hasoc_hi_1637,chants of jai sri ram as owaisi takes oath aim...,NOT,NONE,NONE


## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [4]:
#TODO: implement!

#corpus contains all words, including repeats
corpus = ' '.join([i for i in development['text']]).split()
#all words given assigned a unique value
unique_val = list(pd.factorize(corpus))
#this is only unique words
V = list(set(development['text'].str.split(' ').sum()))
len(V)

1038

In [6]:
V[0:10]


['on', 'चलत', 'समय', 'ahmeds', 'पढ', 'खल', 'लड', 'सगत', 'दरज', 'नरयण']

* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [7]:
#TODO: implement!
#idk if this is right, it returns a vector with a 1 at every location where the word is found
#and zero otherwise


def word_to_one_hot(word):
  one_hot = [0 if word != x else 1 for x in V]
  return one_hot

#X = word_to_one_hot(V[1])
#len(X)
        

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [8]:
#TODO: implement!
def sampling_prob(word):
  wordfreq = 0
  for w in corpus:
    if w == word:
      wordfreq += 1
  return ((math.sqrt(wordfreq/0.001)+1)*(0.001/wordfreq))
  
sampling_prob(V[3])

0.0326227766016838

In [73]:
#s = development["text"].head(1)
s = pd.Index(list(development["text"].head(1)))
s = [x.split() for x in s]
s

TypeError: ignored

# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [66]:
#TODO: implement!

def get_target_context(sentence):
  window_size = 5
  context = []
  indices = [unique_val[word] for word in sentence]
  # when each word is in center
  for center_word_pos in range(len(indices)):
     # for each window position
     for w in range(-window_size, window_size + 1):
       context_word_pos = center_word_pos + w
       # make soure not jump out sentence
       if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
         continue
         context_word_idx = indices[context_word_pos]
         context.append((indices[center_word_pos], context_word_idx))
  return context

#get_target_context(s)
  

TypeError: ignored

# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [9]:
# Set hyperparameters
window_size = 5
embedding_size = 10 #300 is ideal size but lets start small

# More hyperparameters
learning_rate = 0.05
epochs = 100

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [49]:
# Create model 

class Word2Vec(nn.Module):
  def __init__(self):
    super(Word2Vec, self).__init__()
        
    # Hidden layer
    self.W1 = nn.Parameter(torch.randn(embedding_size, len(V), requires_grad=True))

    # Output layer
    self.W2 = nn.Parameter(torch.randn(len(V), embedding_size, requires_grad=True))

  def forward(self, one_hot):
    Z1 = torch.matmul(self.W1, one_hot)
    Z2 = torch.matmul(self.W2, Z1)
    one_hot = F.log_softmax(Z2)
    return one_hot
    

# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [50]:
# Define optimizer and loss
learning_rate = 0.001
model = Word2Vec()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,)
criterion = nn.CrossEntropyLoss()


# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [53]:
# Define train procedure
#One-hot vectors for all unique words
#input = [len(V),len(V)]
i = 0
for i in range(10000):
  for word in V:
    # load initial weights
    output = model(torch.FloatTensor(word_to_one_hot(word)))
    i = i+1
    loss = criterion(output)

    # Backprop to update model parameters 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1) % 50 == 0:
      print ('Step [{}/{}], Loss: {:.4f}'
              .format(i+1, total_step, loss.item()))


#def train():
 
  #print("Training started")

#train()

print("Training finished")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


TypeError: ignored

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).